# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from IFP import IFP, list_questions

In [4]:
ifps = list_questions()['results']

In [5]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[28204, 28205, 28206, 28207, 28208, 28209, 28210]

In [8]:
from Forecaster import Forecaster

In [9]:
forecasters = [Forecaster({id: IFP(id) for id in today_ids}) for i in range(5)]

In [10]:
for fcst in forecasters:
    fcst.fit()

MODEL DOMAIN
28204 Will the Chicago White Sox lose 124 games in the 2024 MLB season, before October 1, 2024?
Sports performance
28205 Will Boeing's aircraft assembly factory workers continuously be on strike, according to the AP's Strikes Hub, through September 30, 2024?
Corporate action
28206 Will a ChatGPT model be ranked #1 overall on the LMSYS Chatbot Arena Leaderboard at the end of the 3rd Quarter of 2024?
AI performance
28207 Will a Gemini model be ranked #1 overall on the LMSYS Chatbot Arena Leaderboard at the end of the 3rd Quarter of 2024?
AI performance
28208 Will a Claude model be ranked #1 overall on the LMSYS Chatbot Arena Leaderboard at the end of the 3rd Quarter of 2024?
AI performance
28209 Will a Grok model be ranked #1 overall on the LMSYS Chatbot Arena Leaderboard at the end of the 3rd Quarter of 2024?
AI performance
28210 Will a Llama model be ranked #1 overall on the LMSYS Chatbot Arena Leaderboard at the end of the 3rd Quarter of 2024?
AI performance
CORRELATOR
|2

In [15]:
for i, fcst in enumerate(forecasters):
    print("forecaster", i)
    fcst.report()
    print("=====================================================================")

forecaster 0
28154 Will Kennedy Iyere win the September 21, 2024 Edo state gubernatorial election in Nigeria?
Forecast 5
Rationale Based on the extensive news and background information, the primary contenders for the September 21, 2024 Edo State gubernatorial election appear to be the candidates from the PDP (Asue Ighodalo), APC (Monday Okpebholo), and LP (Olumide Akpata). Kennedy Iyere, representing the Accord Party, has significantly less visibility and support compared to these leading candidates. The Accord Party's smaller footprint in Nigerian politics overall and the controversy within the party further diminishes his likelihood of winning. Therefore, Kennedy Iyere's chances of winning can be considered very low. 

28155 Will Comex Gold Spot prices exceed $2,700 per ounce before October 1, 2024 according to Yahoo Finance?
Forecast 40
Rationale Gold prices are currently at historic highs, driven by expectations of further interest rate cuts and geopolitical tensions. However, the

In [12]:
import numpy as np
from Summarizer import Summarizer
from MetacGPT import MetacGPT

In [13]:
self = Forecaster({id: IFP(id) for id in today_ids})

In [14]:
editor = Summarizer(MetacGPT)

In [15]:
for ifp in self.ifps.values():
    ifp.forecast = np.mean([fcst.ifps[ifp.id].forecast for fcst in forecasters])
    ifp.rationale = editor.summarize('\n'.join([fcst.ifps[ifp.id].rationale for fcst in forecasters]))

In [16]:
self.report()

28204 Will the Chicago White Sox lose 124 games in the 2024 MLB season, before October 1, 2024?
Forecast 93.0
Rationale Given the Chicago White Sox's current poor performance, losing record, and trends, it is highly probable they will reach 124 losses by the end of the 2024 season. 

28205 Will Boeing's aircraft assembly factory workers continuously be on strike, according to the AP's Strikes Hub, through September 30, 2024?
Forecast 89.0
Rationale Given the overwhelming union support for the strike, dissatisfaction with Boeing’s contract proposal, historical precedents of long strikes, and the substantial demands and complex negotiations, it is highly probable that the strike will continue through September 30, 2024. 

28206 Will a ChatGPT model be ranked #1 overall on the LMSYS Chatbot Arena Leaderboard at the end of the 3rd Quarter of 2024?
Forecast 78.0
Rationale Given the recent improvements and strong track record of the "chatgpt-4o-latest" model, it is likely to maintain its #1 

In [17]:
self.upload()

Prediction posted for  28204
Comment posted for  28204
Prediction posted for  28205
Comment posted for  28205
Prediction posted for  28206
Comment posted for  28206
Prediction posted for  28207
Comment posted for  28207
Prediction posted for  28208
Comment posted for  28208
Prediction posted for  28209
Comment posted for  28209
Prediction posted for  28210
Comment posted for  28210
